In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from datetime import datetime
import numpy as np

In [2]:
spark = SparkSession.builder.appName('hw_2022').master('local[*]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/13 08:06:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/13 08:07:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/13 08:07:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Question 1. Install Spark and PySpark
# Execute spark.version
# What's the output?

spark.version

#### Answer: 3.3.2

'3.3.2'

In [4]:
# Question 2. HVFHW February 2021
# Repartition it to 24 partitions and save it to parquet.
# What's the size of the folder with results (in MB)?

!curl -Lf https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet -o hw_2021.parquet
df = spark.read.option('header', 'true').option('inferSchema', 'true').parquet('hw_2021.parquet')
df.repartition(24).write.parquet('hw_2022')
!du -hs hw_2022/

#### Answer: 516M

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  288M  100  288M    0     0  14.9M      0  0:00:19  0:00:19 --:--:-- 17.2M


[Stage 3:======================================================>  (23 + 1) / 24]

516M	hw_2022/


In [5]:
# Question 3. Count records
# How many taxi trips were there on February 15?
# Consider only trips that started on February 15.


df = df.filter((df.pickup_datetime >= datetime(2021, 2, 1)) & (df.pickup_datetime < datetime(2021, 3, 1)))

filtered = df.withColumn('duration', df.dropoff_datetime - df.pickup_datetime)
filtered.createTempView('hw')
spark.sql(("""
            
            select 
                count(*) as num_trips 
            from hw
            where date(pickup_datetime) = '2021-02-15'
            
            """)).show()

#### Answer: 367170

[Stage 4:============================================>              (3 + 1) / 4]

+---------+
|num_trips|
+---------+
|   367170|
+---------+



In [6]:
# Question 4. Longest trip for each day
# Now calculate the duration for each trip.
# Trip starting on which day was the longest?

spark.sql(("""
            select 
                date(pickup_datetime) as date,
                max(trip_miles) as max_miles
            from hw
            group by 1
            order by 2 desc
            """)).head(1)

#### Answer: 527.11

[Row(date=datetime.date(2021, 2, 18), max_miles=527.11)]

In [7]:
# Question 5. Most frequent dispatching_base_num
# Now find the most frequently occurring dispatching_base_num in this dataset.
# How many stages this spark job has?

spark.sql(("""
            select
                dispatching_base_num,
                count(*) as count
            from hw
            group by 1
            order by 2 desc
            limit 5
            """)).show()

### Answer: 2, one for grouping and one for sorting.

[Stage 10:===========================================>              (3 + 1) / 4]

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+



In [8]:
# Question 6. Most common locations pair
# Find the most common pickup-dropoff pair.
# For example:
# "Jamaica Bay / Clinton East"
# Enter two zone names separated by a slash
# If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East".

!curl -Lf https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv -o z.csv
zone = spark.read.option('header', 'true').option('inferSchema', 'true').csv('z.csv')
zone.createOrReplaceTempView('zone')

spark.sql(("""
            select
                concat(z1.Zone, ' / ', z2.Zone) as zone
            from hw
            join zone z1 on z1.LocationID = hw.PULocationID
            join zone z2 on z2.LocationID = hw.DOLocationID
            group by 1
            order by count(*) desc
            """)).head(1)

#### Answer: East New York / East New York

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0  23172      0 --:--:-- --:--:-- --:--:-- 23172


[Row(zone='East New York / East New York')]

In [9]:
spark.stop()
!rm -rf *.csv *.parquet hw_*